# Projecting embeddings on TensorFlow projector

how you can write the vectors of an embeddings into a TSV file to visualise it in a 3D space on the [TensorFlow projector](https://projector.tensorflow.org/)

In [1]:
##import the required libraries and APIs
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

2.8.0


## Downloading the TensorFlow `imdb_review` dataset

> Make sure tensorflow_datasets is installed

In [2]:
##load the imdb reviews dataset
data, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0NCF6H/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0NCF6H/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## Segregating training and testing sets

In [3]:
# segregate training and test set
train_data, test_data = data['train'], data['test']

##create empty list to store sentences and labels
train_sentences = []
test_sentences = []

train_labels = []
test_labels = []

In [4]:
##iterate over the train data to extract sentences and labels
for s, l in train_data:
  train_sentences.append(s.numpy().decode('utf8'))
  train_labels.append(l.numpy())

##iterate over the test set to extract sentences and labels
for s, l in test_data:
  test_sentences.append(s.numpy().decode('utf8'))
  test_labels.append(l.numpy())

In [6]:
##convert labels lists into numpy array
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

## Data preparation - setting up the tokenizer

In [8]:
##define the parameters for the tokenizing and padding
vocab_size = 10000
oov_token = '<OOV>'
max_length = 120
padding_type = 'post'
truncating_type = 'post'
embedding_dim = 16
dense_dim = 6

In [12]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

##training sequences
train_seqs = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_seqs, maxlen=max_length, padding=padding_type, truncating=truncating_type)

##testing sequences 
test_seqs = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_seqs, maxlen=max_length)

In [14]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(train_sentences[1], '\n')
print(train_padded[1], '\n')
print(decode_review(train_padded[1]))

I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all. 

[  11   26   75  571    6  805 2354  313  106   19   12    7  629  686
    6    4 2219    5  181  584   64 1454  110 2263    3 3951   21    2
    1    3  258   41 4677    4  174  188   21   12 4078   11 1578 2354
   86    2   20   14 1907    2  112  940   14 1811 1340  548    3  355
  181  466    6  591   19   17   55 1817    5   49   14 4044   96   40
  136   11  972   11  201

## Define the Neural Network with Embedding layer

1. Use the Sequential API.
2. Add an embedding input layer of input size equal to vocabulary size.
3. Add a flatten layer, and two dense layers.

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(Flatten())
model.add(Dense(dense_dim, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

##compile the model with loss function, optimizer and metrics
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Model Training

In [17]:
num_epochs = 10

##train the model with training and validation set
model.fit(
    train_padded, 
    train_labels, 
    epochs=num_epochs, 
    validation_data=(test_padded, test_labels)
    )

Epoch 1/10
782/782 [==============================] - 6s 7ms/step - loss: 0.4820 - accuracy: 0.7519 - val_loss: 0.3490 - val_accuracy: 0.8447
Epoch 2/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2392 - accuracy: 0.9076 - val_loss: 0.3769 - val_accuracy: 0.8368
Epoch 3/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0987 - accuracy: 0.9735 - val_loss: 0.4679 - val_accuracy: 0.8189
Epoch 4/10
782/782 [==============================] - 7s 10ms/step - loss: 0.0256 - accuracy: 0.9966 - val_loss: 0.5462 - val_accuracy: 0.8206
Epoch 5/10
782/782 [==============================] - 7s 8ms/step - loss: 0.0072 - accuracy: 0.9993 - val_loss: 0.6198 - val_accuracy: 0.8186
Epoch 6/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.6647 - val_accuracy: 0.8213
Epoch 7/10
782/782 [==============================] - 5s 7ms/step - loss: 9.8679e-04 - accuracy: 1.0000 - val_loss: 0.7209 - val_accuracy: 0.8206
E

## Deriving weights from the embedding layer

In [20]:
# isolating the first embedding layer
layer_0 = model.layers[0]

# extracting learned weights
weights = layer_0.get_weights()[0] # shape: (vocab_size, embedding_dim)

print(weights.shape)
print(weights[0])

(10000, 16)
[-0.00119036  0.01648478  0.00987187  0.03593821  0.00902872  0.01166376
  0.07984091 -0.00640171  0.05244013  0.00248437 -0.02535912 -0.03514944
 -0.004055   -0.01666377 -0.02857905  0.01339901]


## Downloading the vectors and metadata

In [21]:
# import I/O module
import io

# open text stream for vectors
vectors = io.open('vectors.tsv', 'w', encoding='utf-8')

# open text stream for metadata
meta = io.open('meta.tsv', 'w', encoding='utf-8')

# write each word and its corresponding embedding
for index in range(1, vocab_size):
  word = reverse_word_index[index] # flipping the key-value in word_index
  embeddings = weights[index]
  
  meta.write(word + '\n')
  vectors.write('\t'.join([str(x) for x in embeddings]) + '\n')

##close the stream
vectors.close()
meta.close()

In [23]:
# download the written files to local machine
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vectors.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>